# Fine Tune GPT3.5 Turbo Model With Custom Data

Zoumana KEITA

## Data Loading

In [38]:
%%bash
pip -q install datasets

In [4]:
from datasets import load_dataset

In [39]:
# Acquire the training data from Hugging Face
yahoo_answers_qa = load_dataset("yahoo_answers_qa", split="train")

In [40]:
# Check the length of the data and its structure
yahoo_answers_qa

Dataset({
    features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
    num_rows: 87362
})

In [41]:
# Get a subset of 500 observations
yahoo_answers_qa = yahoo_answers_qa.select(range(500))

yahoo_answers_qa

Dataset({
    features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
    num_rows: 500
})

In [42]:
yahoo_answers_qa[0]

{'id': '2020338',
 'question': 'Why did the U.S Invade Iraq ?',
 'answer': "A small group of politicians believed strongly that the fact that Saddam Hussien remained in power after the first Gulf War was a signal of weakness to the rest of the world, one that invited attacks and terrorism. Shortly after taking power with George Bush in 2000 and after the attack on 9/11, they were able to use the terrorist attacks to justify war with Iraq on this basis and exaggerated threats of the development of weapons of mass destruction. The military strength of the U.S. and the brutality of Saddam's regime led them to imagine that the military and political victory would be relatively easy.",
 'nbestanswers': ["A small group of politicians believed strongly that the fact that Saddam Hussien remained in power after the first Gulf War was a signal of weakness to the rest of the world, one that invited attacks and terrorism. Shortly after taking power with George Bush in 2000 and after the attack on 

## Format the data for fine-tuning

For fine-tuning, a consistent format is maintained for each pair of questions and answers across the entire training and testing data.  

{              
  "messages": [             
    {"role": "system", "content": "SYSTEM's ROLE"},           
    {"role": "user", "content": "USER's QUESTION"},            
    {"role": "assistant", "content": "SYSTEM's RESPONSE"}
  ]
}

In [11]:
import random

def format_data(data):

  formatted_data = [
    {"messages": [
        {
            "role": "system",
            "content": "You are the Yahoo platform user's assistant. Please reply users' answer using polite and respectful language."
        },

        {
            'role': 'user',
            'content': message["question"]
        },

        {
            'role': 'assistant',
            'content': message["answer"]
        }]
     } for message in data
  ]

  random.shuffle(formatted_data)

  return formatted_data

In [43]:
formatted_data = format_data(yahoo_answers_qa)

formatted_data[0]

{'messages': [{'role': 'system',
   'content': "You are the Yahoo platform user's assistant. Please reply users' answer using polite and respectful language.spectful language."},
  {'role': 'user', 'content': 'How did the planet Pluto get its name?'},
  {'role': 'assistant',
   'content': 'All the planets were named for Roman Gods. Pluto is named for the Lord of the Underworld because it is the farthest from the sun.'}]}

## Training and Validation Data

In [44]:
# 70% of the overall 500 observation
TRAIN_SIZE = 350

training_data = formatted_data[:TRAIN_SIZE]
validation_data = formatted_data[TRAIN_SIZE:]

print(f"Training Size: {len(training_data)}")
print(f"Validation Size: {len(validation_data)}")

Training Size: 350
Validation Size: 150


## Save the training and validation data

In [46]:
import json

def save_data(dictionary_data, file_name):

  with open(file_name, 'w') as outfile:
    for entry in dictionary_data:
      json.dump(entry, outfile)
      outfile.write('\n')

In [47]:
save_data(training_data, "training_data.jsonl")
save_data(validation_data, "validation_data.jsonl")

In [48]:
%%bash
ls

flagged
sample_data
training_data.jsonl
validation_data.jsonl


## Upload Data to OpenAI

In [22]:
%%bash
pip -q install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 2.1 MB/s eta 0:00:00


In [23]:
import openai
from openai import FineTuningJob, ChatCompletion

openai.api_key = 'YOUR KEY'

In [24]:
def upload_fine_tuning_data(data_path):

  oploaded_file = openai.File.create(
    file=open(data_path, "rb"),
    purpose='fine-tune'
  )

  return oploaded_file

### Training data

In [ ]:
uploaded_training_data = upload_fine_tuning_data("training_data.jsonl")
uploaded_training_data

<File file id=file-dlXBxJVIhZK1ygaDgze6nzZT at 0x79e113ca43b0> JSON: {
  "object": "file",
  "id": "file-dlXBxJVIhZK1ygaDgze6nzZT",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 191790,
  "created_at": 1692937269,
  "status": "uploaded",
  "status_details": null
}

In [ ]:
uploaded_training_id = uploaded_training_data["id"]
print(uploaded_training_id)

file-dlXBxJVIhZK1ygaDgze6nzZT


### Validation data

In [ ]:
uploaded_validation_data = upload_fine_tuning_data("validation_data.jsonl")
uploaded_validation_data

<File file id=file-GZsY8KiXL6Y5OyrOGZMZPQhw at 0x79e0ff7613f0> JSON: {
  "object": "file",
  "id": "file-GZsY8KiXL6Y5OyrOGZMZPQhw",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 80063,
  "created_at": 1692937398,
  "status": "uploaded",
  "status_details": null
}

In [ ]:
uploaded_validation_id = uploaded_validation_data["id"]
print(uploaded_validation_id)

file-GZsY8KiXL6Y5OyrOGZMZPQhw


## Fine-tuning

In [ ]:
def create_fine_tuning(model_name, train_id, val_id):

  fine_tuning_response = FineTuningJob.create(
      training_file=train_id,
      validation_file=val_id,
      model=model_name
  )

  return fine_tuning_response

## Trigger fine-tuning

In [ ]:
model_name = "gpt-3.5-turbo"

fine_tuning_response = create_fine_tuning(model_name,
                                          uploaded_training_id,
                                          uploaded_validation_id)

print(fine_tuning_response)

{
  "object": "fine_tuning.job",
  "id": "ftjob-OLaMvZouQ9rlaHw6EYeSzqjh",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692937704,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-0fNevKdYONDre498IjKAHt12",
  "result_files": [],
  "status": "created",
  "validation_file": "file-GZsY8KiXL6Y5OyrOGZMZPQhw",
  "training_file": "file-dlXBxJVIhZK1ygaDgze6nzZT",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}


In [ ]:
fine_tuning_job_ID = fine_tuning_response['id']

print(fine_tuning_job_ID)

ftjob-OLaMvZouQ9rlaHw6EYeSzqjh


In [ ]:
# Check the state of a fine-tune
fine_tuning_response = FineTuningJob.retrieve(fine_tuning_job_ID)
fine_tuning_response

<FineTuningJob fine_tuning.job id=ftjob-OLaMvZouQ9rlaHw6EYeSzqjh at 0x79e0fcc1ce50> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-OLaMvZouQ9rlaHw6EYeSzqjh",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692937704,
  "finished_at": 1692939142,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::7rJ5vdvh",
  "organization_id": "org-0fNevKdYONDre498IjKAHt12",
  "result_files": [
    "file-x3B3AIFQdVV0TQ9r6pkPcz8H"
  ],
  "status": "succeeded",
  "validation_file": "file-GZsY8KiXL6Y5OyrOGZMZPQhw",
  "training_file": "file-dlXBxJVIhZK1ygaDgze6nzZT",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 109191
}

In [ ]:
fine_tuned_model_ID = fine_tuning_response["fine_tuned_model"]

## Inferencing of the Fine Tuned Model

In [26]:
def answer_question(question, model_ID=fine_tuned_model_ID):

  message = [
              {
                  "role": "system",
                  "content": "You are the Yahoo platform user's assistant. Please reply users' answer using polite and respectful language.spectful language."
              },

              {
                  "role": "user",
                  "content": question
              }
            ]

  # Start inferencing
  model_completion = ChatCompletion.create(model=model_ID, messages=message)

  # Get the response
  response = model_completion.choices[0].message

  return response["content"]


In [27]:
import textwrap
wrapper = textwrap.TextWrapper(width=60)

In [28]:
question = "Which Internet browser is the best and why?"

### Using the fine-tuned model

In [49]:
response = answer_question(question, model_ID=fine_tuned_model_ID)

In [50]:
print(wrapper.fill(response))

All mozillas are good and one good browser of mozilla is
netscape. Opera is quite good too and since it is lighter it
runs faster.. Mozilla and opera rock!. I have both( 3
versions and mozilla separetely)


### Using the GPT3.5 turbo model

In [51]:
response = answer_question(question, model_ID="gpt-3.5-turbo")

In [52]:
print(wrapper.fill(response))

The best internet browser depends on personal preferences
and individual needs. Some popular choices include Google
Chrome, Mozilla Firefox, Microsoft Edge, and Safari. Each
browser has different features and advantages. It is
recommended to try different browsers and see which one fits
your requirements in terms of speed, security, user
interface, and compatibility with websites and extensions.


## Graphical Interface Using Gradio

In [35]:
!pip -q install gradio

In [36]:
import gradio as gr

In [37]:
title = "Yahoo Forum User Assistant"

description = (
    "The answer to the question is given based on the chosen model"
    )

UI = gr.Interface(
    title = title,
    description=description,
    fn = answer_question,
    inputs="text",
    outputs="text"
)

UI.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a9ba836cbe6fa8f0ef.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
